In [1]:
# код для того, чтобы получить доступ к папке с модулями ПДН, да и в принципе в любую папку по пути
# в принципе можно бы прописать функцией для импорта, чтобы передавать этой функции просто путь модулей(корневую папку)

In [2]:
import sys
path_to_pdn = r"C:\Users\murad_satabaev\Documents\GitHub\pdn_" # сюда внутри кавычек вставляем путь, где ледит репозиторий пдн
sys.path.insert(0, path_to_pdn)

In [3]:
def insert_values_from_data_to_ecxel_pattern(path_to_pattern: str, df: pd.DataFrame, path_to_save, sheet_name: str='Лист1',
                                             start_column: int=1, start_row: int=1):
    """
    Функция вставляет значения датафрэйма в шаблон xlsx-файла, не меняя при этом формат ячеек и колонок,
    """
    from openpyxl.utils.dataframe import dataframe_to_rows
    from openpyxl import load_workbook
    workbook = load_workbook(path_to_pattern)
    sheet_by_name = workbook[sheet_name]
    rows = dataframe_to_rows(df, index=False, header=False)
    for r_idx, row in enumerate(rows, start_row):
        for c_idx, value in enumerate(row, start_column):
             sheet_by_name.cell(row=r_idx, column=c_idx, value=value)
    workbook.save(path_to_save)

NameError: name 'pd' is not defined

In [ ]:
def delete_rows(path_to_pattern: str, path_to_save, sheet_name: str='Лист1',
                                            start_row: int=1,  last_row: int=1):
    """
    
    """
    from openpyxl.utils.dataframe import dataframe_to_rows
    from openpyxl import load_workbook
    workbook = load_workbook(path_to_pattern)
    sheet_by_name = workbook[sheet_name]
    num_row = last_row - start_row + 1
    sheet_by_name.delete_rows(start_row, num_row)
    workbook.save(path_to_save)

In [ ]:
def get_google_sheet_list_as_df(gs_url, sheet_num=0):
    """
    Скачивает датафрэйм по номеру листа в Google Drive
    !!! Важно !!!
    Нужен доступ к скачиваемому файлу почте murad-464@murad-satbi-1st-pro.iam.gserviceaccount.com
    """
    import os
    import gspread
    import gspread_dataframe as gd
    from gspread_dataframe import get_as_dataframe
    json_path = os.path.join(rf'\\t999', 'Сетевой диск', 'pdn', 'configs', 'google_sheets.json')
    gc = gspread.service_account(filename=json_path)
    sh = gc.open_by_url(gs_url)
    ws = sh.get_worksheet(sheet_num)
    df = gd.get_as_dataframe(ws)
    return df

In [ ]:
def assistants_present_and_absent_df(df, to_return: str='df'):    
    """
    Принимает датафрэйм с помощниками врачами и их присутствием или отсуствием
    Возвращает в зависимости от значения to_return:
    'df' - весь датафрэйм
    'absent' - список логинов пдн отсутствующих
    'present' - количество помощников на смене
    'total' - общее количество помощников
    """
    df1 = df.iloc[3:100, :7]
    df1 = df1.dropna(how='all')
    df1 = df1.set_axis(range(7), 1).reset_index(drop=True).drop(columns=[0, 2])
    df1 = df1.set_axis(['hall_number', 'full_name', 'login', 'mo', 'present'], 1)
    rows_to_delete = set(df1[df1['hall_number'] == 'Зал №'].index)
    df1 = df1.drop(rows_to_delete)
    df1 = df1.fillna(0).reset_index(drop=True)
    absent_pdn_list = set(df1[df1.present == 1].login)
    present = df1[df1.present == 0].login.count()
    total = len(absent_pdn_list) + present
    returnables = dict()
    returnables['df'] = df1
    returnables['absent'] = absent_pdn_list
    returnables['present'] = present
    returnables['total'] = total
    return returnables[to_return]

In [ ]:
def assistants_distribution(df):
    df1 = df.iloc[:, :6]
    df1 = df1.dropna(how='all')
    df1 = df1.rename(columns={
    'Наименование МО':'mo_name',
    'ФИО врача':'doc_name',
    'Количество населения на ДН':'dn_population',
    'группа':'group',
    'ФИО помощника врача':'assistant_name',
    'Логин помощника врача':'login'})
    df1 = df1[~df1.login.isna()].reset_index(drop=True)
    df1[['dn_population', 'group']] = df1[['dn_population', 'group']].astype(int)
    return df1